In [ ]:
import scipy.stats as sps
import numpy as np
from CMF_Predictor import CMFRecommender
import pandas as pd
import math
import random
import sklearn

from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm

from libmf import mf

In [ ]:
def usefullness(c, w, noice):
    return np.arctan(c - w + noice) / np.pi + 0.5

In [ ]:
customer_distribution = sps.norm(0.6, 0.2)
w_distribution = sps.norm(0, 0.4)

epsilon = sps.norm(0, 0.05)

In [ ]:
def zero_step(model: CMFRecommender, user_info, item_info, topn=5):
    new_feedback = []

    w_offered = model.recommend_items_new(user_id, item_info["F"], topn=topn)  # вот тут придумать, какие user_id будем подставлять 
    for w in w_offered:
        deal = sps.bernoulli.rvs(usefullness(user_row["F"], w, epsilon.rvs()))  # моделируем сделки
        new_feedback.append((user_row["UserId"], w, deal))
    
    for index, user_row in user_info.iterrows():
        w_offered = model.recommend_items_cold(user_row["F"], topn)["ItemId"]
        for w in w_offered:
            deal = sps.bernoulli.rvs(usefullness(user_row["F"], w, epsilon.rvs()))  # моделируем сделки
            new_feedback.append((user_row["UserId"], w, deal))
    
    
    model.retrain(new_feedback, user_info, item_info)
    return model


In [ ]:
def dynamic_system_iterate(model: CMFRecommender, customer_distribution, w_distribution, c_size, w_size, num_of_steps=5,
                           topn=5):
    
    maximal_user, maximal_item = model.get_max_index()
    user_info = pd.DataFrame({"F": customer_distribution.rvs(size=c_size)})  # size = (c_size, c_feature_size) в многомерном случае 
    user_info["UserId"] = np.arange(maximal_user + 1, maximal_user + 1  + c_size)

    item_info = pd.DataFrame({"F": w_distribution.rvs(size=w_size)})  # size = (w_size, w_feature_size) в многомерном случае 
    item_info["ItemId"] = np.arange(maximal_item + 1, maximal_item + 1 + w_size)

    model = zero_step(model, user_info, item_info, topn=topn)
    
    new_feedback = []
    for step in range(1, num_of_steps + 1):

        for index, user_row in user_info.iterrows():
            w_offered = model.recommend_items(user_row["UserId"], topn=topn, exclude_rated=True)["ItemId"]
            for w in w_offered:
                deal = sps.bernoulli.rvs(usefullness(user_row["F"], w, epsilon.rvs()))  # моделируем сделки
                new_feedback.append((user_row["UserId"], w, deal))


    ## сделать фит в scipy.stats.rv_continuous, пока что хз как( и в return уже новое распределение
    return customer_distribution, w_distribution, new_feedback